In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fedai.FLearner import *  # noqa: F403
from fedai.learner_utils import *  # noqa: F403
from fedai.utils import * # noqa: F403
from omegaconf import OmegaConf

def cfg_fn(f):
    cfg = load_config(f)  # noqa: F405
    cfg = OmegaConf.create(cfg)
    return cfg
cfg = cfg_fn('cfg.yaml')

In [ ]:
import argparse
args = argparse.Namespace(**cfg)

In [ ]:
args.project_name

'example_project'

In [ ]:
learner = FLearner(cfg, client_fn= client_fn)  # noqa: F405


In [ ]:
# learner.run_simulation()

data/MNIST/train data/MNIST/test

Dataset already generated.

data/MNIST/train data/MNIST/test

Dataset already generated.



  0%|          | 0/484 [00:00<?, ?it/s]

KeyboardInterrupt: 